In [ ]:
### NOTES ###
# This code is far from perfect. It works well enough for the purpose it was designed for.
# However, I am far from the best programmer, and am far more comfortable in R than Python. 
# That is all just to say that this is not text book, but I hope it is good enough to help someone scrape some data.
# If you do see errors here or in the other scripts, please reach out to me at matt.meister@colorado.edu

## The best way to use this to learn: ##
# Would be to take code out of the for loops it is in, and run things piece by piece
# Do so with the REI website up alongside, preferably while able to see the source code (i.e., right click "inspect")

## Citing ##
# If you use this code or data in a way that feels necessary to cite, please do so as:
# Meister, M., & Reinholtz, N. (2022). Quality in Context: Evidence that Consumption Context Influences User-Generated Product Ratings. Available at SSRN 4155522.
# The most updated version of that paper can be found here: https://drive.google.com/file/d/1BfDzIxTsCtQOMRwkbAlK12xDKWDXSbSN/view?usp=sharing

In [1]:
# Start by importing necessary stuff
import json # A lot of data online is stored as jsons
import pandas as pd
import re # Regular expressions, to help us parse html, json.
import requests # To pull internet pages
import time # To allow us to "sleep" the scraping, so as not to overwhelm the servers
import os # To create folders
from bs4 import BeautifulSoup # To make html easier to parse

In [2]:
products = [] # We'll put products in here
larger_categories = [] # This is where we'll put overarching categories. 
# I don't want them, since they'll lead to a lot of duplicate scraping

# Scrape categories
html = BeautifulSoup(requests.get('https://www.rei.com/categories').text, 'html.parser')

# Find overarchign categories
larger_list = html.find_all('div', class_="all-categories__parent-header")
larger_categories = ["https://www.rei.com" + tag.find('a').get('href','') if tag.find('a') else '' for tag in larger_list]


# Find all categories
product_list = html.find_all('a', class_='cdr-link_13-0-3 cdr-link--standalone_13-0-3')
products = ["https://www.rei.com" + d['href'] for d in product_list]

# Remove overarching categories
products = [x for x in products if x not in larger_categories]

# Set up the rest of the scraping!
# Set the base url to call reviews from their hosting site
url = 'https://api.bazaarvoice.com/data/batch.json'
# Give yourself something to not look like a bot
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15'}
    
page = 1 # This is for navigation later
bags = [] # This will become a list of links for all products in a category
pids = [] # This will become a list of links for all product IDs in a category
prices = [] # This will become a list of links for all prices in a category

os.mkdir(f'PRICES') #To store price data 
#NOTE: This ^ is a bad way to store prices, but it's fine for my purpose. 
# You might prefer storing prices in the OVRs files.
os.mkdir(f'JSONs') #To store JSON data for all products
os.mkdir(f'QUESTIONS') #To store questions data
os.mkdir(f'REVIEWS') #To store reviews data
os.mkdir(f'OVRs') #To store overall product info data (summarizing the JSONs)

for product in products:
    os.mkdir(f'REI_{product}') # Make folders for each product category. Note that this is personal preference
    os.mkdir(f'REI_{product}/REI_{product}_products')
    os.mkdir(f'REI_{product}/REI_{product}_questions')
    os.mkdir(f'REI_{product}/REI_{product}_reviews')
    os.mkdir(f'REI_{product}/REI_{product}_ovr')
    page = 1
    bags = []
    pids = []
    prices = []
    
    # Get links for all results pages
    html = BeautifulSoup(requests.get(f'https://www.rei.com/c/{product}?pagesize=90').text, 'html.parser')
    time.sleep(1)
    num_pg = 1 if html.find('a', class_='e98NTAHiDGsqQqLsoyvz-') == None else html.find('a', class_='e98NTAHiDGsqQqLsoyvz-')['href'][-1]
    pages = list(range(1,int(num_pg)+1))
    
    # Go through each page, scrape all product links
    for page in pages:
        html = BeautifulSoup(requests.get(f'https://www.rei.com/c/{product}?pagesize=90&page={page}').text, 'html.parser')
        bags = bags + ['https://www.rei.com' + d['href'] for d in html.find_all('a', class_="_1A-arB0CEJjk5iTZIRpjPs _1K5N3WSl_8ywawYr0tzSgT")]
    
    #Fun fact: this project first started with me scraping sleeping bags. The name 'bag' is leftover from that.
    # Get the product IDs out of links
    for bag in bags:
        pids.append(re.findall(r"(\d{6})", bag)[0]) 
    
    # I am setting all outlet items to be priced at-99, since I assume their price has dropped significantly recently (ish)
    # I'll set prices here
    for bag in bags:
        if bag[20:30] == 'rei-garage':
            prices.append('-99')
        else:
            d=requests.get(bag).text
            time.sleep(2)
            html = BeautifulSoup(d, 'html.parser')
            price = re.findall(r'(\d{0,5}\.\d{1,2})', str(html.find_all('span', class_='price-value')))
            if price == []:
                prices.append(-99)
            else:
                prices.append(price[0])

    # Create the data frame of prices for each product, and write it. Note that it is hilariously small.
    # Again, you might prefer doing prices differently. 
    pricesDF = pd.DataFrame({'pid':pids,
                            'price':prices})

    pricesDF.to_csv(f'PRICES/REI_{product}_prices.csv')
    
    # Now the good stuff! Call the API to get reviews, questions, other stuff
    for pid in pids:
        params = {
            # TO GET PARAMETER INFO:
            # 1) GO TO A PRODUCT ON REI.COM
            # 2) RIGHT CLICK
            # 3) INSPECT ELEMENT (ON SAFARI)
            # 4) GO TO NETWORK > BATCH.JSON > HEADERS
        'passkey': '',#This changes semi-frequently
        'apiversion': '5.5',
        'displaycode': '15372-en_us',
        'resource.q0': 'products',
        'filter.q0': f'id:eq:{pid}',
        'stats.q0': 'questions,reviews',
        'filteredstats.q0': 'questions,reviews',
        'filter_questions.q0': 'contentlocale:eq:en*,en_US',
        'filter_answers.q0': 'contentlocale:eq:en*,en_US',
        'filter_reviews.q0': 'contentlocale:eq:en*,en_US',
        'filter_reviewcomments.q0': 'contentlocale:eq:en*,en_US',
        'resource.q1': 'questions',
        'filter.q1': f'productid:eq:{pid}',
        'sort.q1': 'totalanswercount:desc',
        'stats.q1': 'questions',
        'filteredstats.q1': 'questions',
        'include.q1': 'authors,products,answers',
        'filter_questions.q1': 'contentlocale:eq:en*,en_US',
        'filter_answers.q1': 'contentlocale:eq:en*,en_US',
        'limit.q1': '20',
        'offset.q1': '0',
        'limit_answers.q1': '10',
        'resource.q2': 'reviews',
        'filter.q2': f'productid:eq:{pid}',
        'sort.q2': 'submissiontime:desc',
        'stats.q2': 'reviews',
        'filteredstats.q2': 'reviews',
        'include.q2': 'authors,products,comments',
        'filter_reviews.q2': 'contentlocale:eq:en*,en_US',
        'filter_reviewcomments.q2': 'contentlocale:eq:en*,en_US',
        'filter_comments.q2': 'contentlocale:eq:en*,en_US',
        'limit.q2': '100',
        'offset.q2': '0',
        'limit_comments.q2': '20',
        'callback': 'JSONPHandler'
    }
        #Get that stuff!
        req = requests.get(url, headers = headers, params = params)
        data_json = req.text.split("(", 1)[1].strip(")") # Convert to json
        batched_results = json.loads(data_json)['BatchedResults']
        
        # Write the raw JSON so you can use it later if needed
        if batched_results['q0']['Results']==[]:
            continue
        products_json = batched_results['q0']['Results'][0]
        with open(f'JSONs/REI_{pid}_batched_results.json', 'w') as outfile:
            json.dump(products_json, outfile)
        
        # Scrape questions to use if desired
        questions = pd.DataFrame(batched_results['q1']['Results'])
        questions.to_csv(f'QUESTIONS/REI_{pid}_questions.csv')  # Write them as csv
        
        # Scrape reviews
        reviews = pd.DataFrame(batched_results['q2']['Results'])
        reviews.to_csv(f'REVIEWS/REI_{pid}_reviews.csv')  
        
        # Product overall information
        ovr = pd.DataFrame({'Name': 'NA' if products_json['Name'] == {} else[products_json['Name']],
                            'Active': 'NA' if products_json['Active'] == {} else[products_json['Active']],
                            'Brand': 'NA' if products_json['Brand'] == {} else [products_json['Brand']['Id']],
                            'reviews': 'NA' if products_json['TotalReviewCount'] == {} else [products_json['TotalReviewCount']],
                            'pid': f'{pid}', 
                            'ovr': 'NA' if products_json['ReviewStatistics']['AverageOverallRating'] == {} else[products_json['ReviewStatistics']['AverageOverallRating']],
                           'desc': 'NA' if products_json['Description'] == {} else [products_json['Description']],
                           'ProductPageUrl': 'NA' if products_json['ProductPageUrl'] == {} else [products_json['ProductPageUrl']],
                            'category': f'{product}',
                           })
        ovr.to_csv(f'OVRs/REI_{pid}_ovr.csv')
        time.sleep(15) #Rest, so that you don't overload REI